# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f63f8216be0>
├── 'a' --> tensor([[ 0.9099,  0.8847,  1.4803],
│                   [-0.0019, -0.1528, -0.4568]])
└── 'x' --> <FastTreeValue 0x7f63f8216b80>
    └── 'c' --> tensor([[ 1.0141, -0.1235, -0.9643, -0.7970],
                        [ 0.3444, -0.8224,  0.3015,  0.3680],
                        [ 1.8830, -0.0592, -1.2137, -0.6014]])

In [4]:
t.a

tensor([[ 0.9099,  0.8847,  1.4803],
        [-0.0019, -0.1528, -0.4568]])

In [5]:
%timeit t.a

60.8 ns ± 0.0436 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f63f8216be0>
├── 'a' --> tensor([[ 1.3572,  0.7325,  0.7715],
│                   [-0.1579, -0.6597, -0.1823]])
└── 'x' --> <FastTreeValue 0x7f63f8216b80>
    └── 'c' --> tensor([[ 1.0141, -0.1235, -0.9643, -0.7970],
                        [ 0.3444, -0.8224,  0.3015,  0.3680],
                        [ 1.8830, -0.0592, -1.2137, -0.6014]])

In [7]:
%timeit t.a = new_value

64.8 ns ± 0.0527 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9099,  0.8847,  1.4803],
               [-0.0019, -0.1528, -0.4568]]),
    x: Batch(
           c: tensor([[ 1.0141, -0.1235, -0.9643, -0.7970],
                      [ 0.3444, -0.8224,  0.3015,  0.3680],
                      [ 1.8830, -0.0592, -1.2137, -0.6014]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9099,  0.8847,  1.4803],
        [-0.0019, -0.1528, -0.4568]])

In [11]:
%timeit b.a

56.5 ns ± 0.00774 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.0022,  0.2852, -1.4033],
               [ 1.5627,  1.6598, -0.2629]]),
    x: Batch(
           c: tensor([[ 1.0141, -0.1235, -0.9643, -0.7970],
                      [ 0.3444, -0.8224,  0.3015,  0.3680],
                      [ 1.8830, -0.0592, -1.2137, -0.6014]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.134 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

856 ns ± 0.585 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 32.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 224 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f63f821ff10>
├── 'a' --> tensor([[[ 0.9099,  0.8847,  1.4803],
│                    [-0.0019, -0.1528, -0.4568]],
│           
│                   [[ 0.9099,  0.8847,  1.4803],
│                    [-0.0019, -0.1528, -0.4568]],
│           
│                   [[ 0.9099,  0.8847,  1.4803],
│                    [-0.0019, -0.1528, -0.4568]],
│           
│                   [[ 0.9099,  0.8847,  1.4803],
│                    [-0.0019, -0.1528, -0.4568]],
│           
│                   [[ 0.9099,  0.8847,  1.4803],
│                    [-0.0019, -0.1528, -0.4568]],
│           
│                   [[ 0.9099,  0.8847,  1.4803],
│                    [-0.0019, -0.1528, -0.4568]],
│           
│                   [[ 0.9099,  0.8847,  1.4803],
│                    [-0.0019, -0.1528, -0.4568]],
│           
│                   [[ 0.9099,  0.8847,  1.4803],
│                    [-0.0019, -0.1528, -0.4568]]])
└── 'x' --> <FastTreeValue 0x7f6354b29460>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 40.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f63fd2a5c40>
├── 'a' --> tensor([[ 0.9099,  0.8847,  1.4803],
│                   [-0.0019, -0.1528, -0.4568],
│                   [ 0.9099,  0.8847,  1.4803],
│                   [-0.0019, -0.1528, -0.4568],
│                   [ 0.9099,  0.8847,  1.4803],
│                   [-0.0019, -0.1528, -0.4568],
│                   [ 0.9099,  0.8847,  1.4803],
│                   [-0.0019, -0.1528, -0.4568],
│                   [ 0.9099,  0.8847,  1.4803],
│                   [-0.0019, -0.1528, -0.4568],
│                   [ 0.9099,  0.8847,  1.4803],
│                   [-0.0019, -0.1528, -0.4568],
│                   [ 0.9099,  0.8847,  1.4803],
│                   [-0.0019, -0.1528, -0.4568],
│                   [ 0.9099,  0.8847,  1.4803],
│                   [-0.0019, -0.1528, -0.4568]])
└── 'x' --> <FastTreeValue 0x7f6354b1e490>
    └── 'c' --> tensor([[ 1.0141, -0.1235, -0.9643, -0.7970],
                        [ 0.3444, -0.8224,  0.3015,  0.3680],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 73.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.6 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.9099,  0.8847,  1.4803],
                [-0.0019, -0.1528, -0.4568]],
       
               [[ 0.9099,  0.8847,  1.4803],
                [-0.0019, -0.1528, -0.4568]],
       
               [[ 0.9099,  0.8847,  1.4803],
                [-0.0019, -0.1528, -0.4568]],
       
               [[ 0.9099,  0.8847,  1.4803],
                [-0.0019, -0.1528, -0.4568]],
       
               [[ 0.9099,  0.8847,  1.4803],
                [-0.0019, -0.1528, -0.4568]],
       
               [[ 0.9099,  0.8847,  1.4803],
                [-0.0019, -0.1528, -0.4568]],
       
               [[ 0.9099,  0.8847,  1.4803],
                [-0.0019, -0.1528, -0.4568]],
       
               [[ 0.9099,  0.8847,  1.4803],
                [-0.0019, -0.1528, -0.4568]]]),
    x: Batch(
           c: tensor([[[ 1.0141, -0.1235, -0.9643, -0.7970],
                       [ 0.3444, -0.8224,  0.3015,  0.3680],
                       [ 1.8830, -0.0592, -1.2137, -0.6014]],
         

In [26]:
%timeit Batch.stack(batches)

79.2 µs ± 307 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.9099,  0.8847,  1.4803],
               [-0.0019, -0.1528, -0.4568],
               [ 0.9099,  0.8847,  1.4803],
               [-0.0019, -0.1528, -0.4568],
               [ 0.9099,  0.8847,  1.4803],
               [-0.0019, -0.1528, -0.4568],
               [ 0.9099,  0.8847,  1.4803],
               [-0.0019, -0.1528, -0.4568],
               [ 0.9099,  0.8847,  1.4803],
               [-0.0019, -0.1528, -0.4568],
               [ 0.9099,  0.8847,  1.4803],
               [-0.0019, -0.1528, -0.4568],
               [ 0.9099,  0.8847,  1.4803],
               [-0.0019, -0.1528, -0.4568],
               [ 0.9099,  0.8847,  1.4803],
               [-0.0019, -0.1528, -0.4568]]),
    x: Batch(
           c: tensor([[ 1.0141, -0.1235, -0.9643, -0.7970],
                      [ 0.3444, -0.8224,  0.3015,  0.3680],
                      [ 1.8830, -0.0592, -1.2137, -0.6014],
                      [ 1.0141, -0.1235, -0.9643, -0.7970],
                      [ 0.3444, -

In [28]:
%timeit Batch.cat(batches)

141 µs ± 394 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 891 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
